## Pré-print Repositories

In [1]:
import requests
import numpy as np
from pprint import pprint
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [2]:
def scrape_subject_areas(url):
    # Enviar uma solicitação GET para a página
    response = requests.get(url)

    # Dicionário para armazenar as áreas de assunto
    subject_areas_dict = {}

    # Verifique se a solicitação foi bem-sucedida
    if response.status_code == 200:
        # Analise o conteúdo da página com BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Encontre o elemento div que contém as áreas de assunto
        subject_areas_div = soup.find('div', class_='item-list highwire-views-col-list highwire-views-cols-3 clearfix')

        # Verifique se o elemento foi encontrado
        if subject_areas_div:
            # Encontre todas as tags 'a' dentro do div
            subject_links = subject_areas_div.find_all('a')
            
            # Itere sobre as tags 'a' para extrair as áreas de assunto e seus URLs
            for link in subject_links:
                area_name = link.text.strip()
                area_url = link['href']
                subject_areas_dict[area_name] = area_url

        # Exiba o dicionário resultante
        # print(subject_areas_dict)

    return subject_areas_dict

In [3]:
url_bioRxiv = 'https://www.biorxiv.org/'
areas_dict = scrape_subject_areas(url_bioRxiv)
print(f'{len(areas_dict)} coleções de áreas')

if areas_dict:
    for area, area_url in areas_dict.items():
        print(f'   Área: {area:40} URL: {area_url}')
else:
    print('Seção de áreas de assunto não encontrada.')

27 coleções de áreas
   Área: Animal Behavior and Cognition            URL: /collection/animal-behavior-and-cognition
   Área: Biochemistry                             URL: /collection/biochemistry
   Área: Bioengineering                           URL: /collection/bioengineering
   Área: Bioinformatics                           URL: /collection/bioinformatics
   Área: Biophysics                               URL: /collection/biophysics
   Área: Cancer Biology                           URL: /collection/cancer-biology
   Área: Cell Biology                             URL: /collection/cell-biology
   Área: Clinical Trials                          URL: /collection/clinical-trials
   Área: Developmental Biology                    URL: /collection/developmental-biology
   Área: Ecology                                  URL: /collection/ecology
   Área: Epidemiology                             URL: /collection/epidemiology
   Área: Evolutionary Biology                     URL: /collection/evol

In [4]:
def fetch_content_summary_statistics(interval='m', format_='json'):
    # Construa a URL do endpoint
    url = f'https://api.biorxiv.org/sum/{interval}/{format_}'
    
    # Faça a solicitação GET
    response = requests.get(url)
    
    # Verifique se a solicitação foi bem-sucedida
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Erro ao buscar estatísticas de resumo de conteúdo. Código de status: {response.status_code}")
        return None

def fetch_usage_summary_statistics(interval='m', format_='json'):
    # Construa a URL do endpoint
    url = f'https://api.biorxiv.org/usage/{interval}/{format_}'
    
    # Faça a solicitação GET
    response = requests.get(url)
    
    # Verifique se a solicitação foi bem-sucedida
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Erro ao buscar estatísticas de uso. Código de status: {response.status_code}")
        return None

In [5]:
# Exemplo de uso das funções
content_summary_data = fetch_content_summary_statistics()
usage_summary_data   = fetch_usage_summary_statistics()

In [6]:
# Exibir os resultados
# print("Estatísticas de Resumo de Conteúdo:")
# pprint(content_summary_data, width=125)

In [7]:
# print("Estatísticas de Uso:")
# pprint(usage_summary_data, width=125)

In [8]:
def plot_usage_statistics(data, width=1800, height=800):
    # Extrair os valores relevantes dos dados
    months = [entry['month'] for entry in data['bioRxiv content statistics']]
    abstract_cumulative = [np.round(int(entry['abstract_cumulative'])/1000000, 2) for entry in data['bioRxiv content statistics']]
    abstract_views = [np.round(int(entry['abstract_views'])/1000000, 2) for entry in data['bioRxiv content statistics']]

    # Criar um DataFrame com os dados
    import pandas as pd
    df = pd.DataFrame({'Mês': months, 'Resumo Cumulativo': abstract_cumulative, 'Visualizações de Resumo': abstract_views})

    # Criar um gráfico com dois eixos y
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Adicionar barras para 'Resumo Cumulativo'
    fig.add_trace(go.Bar(x=df['Mês'], y=df['Resumo Cumulativo'], name='Resumo Cumulativo', text=df['Resumo Cumulativo'], textposition='outside'))

    # Adicionar linhas para 'Visualizações de Resumo'
    fig.add_trace(go.Scatter(x=df['Mês'], y=df['Visualizações de Resumo'], mode='lines+markers+text', name='Visualizações de Resumo', textposition='top center', yaxis='y2'))

    # Configurar eixos
    max_value_primary = max(df['Resumo Cumulativo'].max()+30, df['Visualizações de Resumo'].max())
    ratio = 50  # Proporção desejada
    max_value_secondary = max_value_primary / ratio

    fig.update_layout(
        title='Estatísticas de uso do bioRxiv',
        xaxis_title='Mês',
        yaxis_title='Milhões de resumos acumulados',
        yaxis2_title='Milhões de visualizações por mês',
        legend=dict(x=0.02, y=0.96),
        width=width,
        height=height,
        yaxis=dict(range=[0, max_value_primary]),
        yaxis2=dict(range=[0, max_value_secondary])
    )

    # Configurar o tamanho da fonte para os rótulos de dados
    fig.update_traces(
        textfont=dict(size=24),  # Tamanho da fonte para os rótulos de dados
        selector=dict(type='bar', name='Resumo Cumulativo')  # Seletor para as barras
    )

    fig.update_traces(
        textfont=dict(size=40),  # Tamanho da fonte para os rótulos de dados
        selector=dict(type='scatter', name='Visualizações de Resumo')  # Seletor para as linhas
    )

    # Exibir o gráfico
    fig.show()

def plot_content_statistics(content_data, width=1800, height=800):
    # Extrair os valores relevantes dos dados
    months = [entry['month'] for entry in content_data['bioRxiv content statistics']]
    new_papers = [entry['new_papers'] for entry in content_data['bioRxiv content statistics']]
    revised_papers = [entry['revised_papers'] for entry in content_data['bioRxiv content statistics']]
    new_papers_cumulative = [entry['new_papers_cumulative'] for entry in content_data['bioRxiv content statistics']]
    revised_papers_cumulative = [entry['revised_papers_cumulative'] for entry in content_data['bioRxiv content statistics']]

    # Criar um DataFrame com os dados
    import pandas as pd
    df = pd.DataFrame({'Mês': months,
                        'Artigos novos no mês': new_papers, 
                        'Artigos revisados no mês':revised_papers, 
                        'Total de Artigos acumulado':new_papers_cumulative , 
                        'Total de Artigos revisados':revised_papers_cumulative})

    # Criar um gráfico com dois eixos y
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Adicionar barras para total acumulado de artigos
    fig.add_trace(go.Bar(x=df['Mês'], y=df['Total de Artigos acumulado'], name='Total de Artigos acumulado', text=df['Total de Artigos acumulado'], textposition='outside'))
    fig.add_trace(go.Bar(x=df['Mês'], y=df['Total de Artigos revisados'], name='Total de Artigos revisados', text=df['Total de Artigos revisados'], textposition='outside'))

    # Configurar eixo principal
    fig.update_yaxes(title_text='Quantidade de Artigos', secondary_y=False)

    # Adicionar linhas para versões acumuladas no eixo secundário
    fig.add_trace(go.Scatter(x=df['Mês'], y=df['Artigos novos no mês'], mode='lines+markers+text', name='Artigos novos no mês', textposition='top center', yaxis='y2'))
    fig.add_trace(go.Scatter(x=df['Mês'], y=df['Artigos revisados no mês'], mode='lines+markers+text', name='Artigos revisados no mês', textposition='top center', yaxis='y2'))

    # Configurar eixos
    max_value_primary = max(df['Total de Artigos acumulado'].max()+30, df['Total de Artigos revisados'].max())
    ratio = 50  # Proporção desejada
    max_value_secondary = max_value_primary / ratio

    fig.update_layout(
        title='Estatísticas de conteúdo no bioRxiv',
        xaxis_title='Mês',
        yaxis_title='Quantidade acumulada de artigos',
        yaxis2_title='Quantidades de artigos em cada mês',
        legend=dict(x=0.02, y=0.96),
        width=width,
        height=height,
        yaxis=dict(range=[0, max_value_primary]),
        yaxis2=dict(range=[0, max_value_secondary])
    )

    # Configurar o tamanho da fonte para os rótulos de dados
    fig.update_traces(
        textfont=dict(size=24),
        selector=dict(type='bar', name='Quantidade de novos papers')  # Seletor para as barras
    )

    fig.update_traces(
        textfont=dict(size=40),
        selector=dict(type='scatter', name='Quantidade de papers revisados')  # Seletor para as linhas
    )
    
    # fig.update_yaxes(overlaying='y2', side='right')

    # Exibir o gráfico
    fig.show()

In [9]:
plot_content_statistics(content_summary_data)
plot_usage_statistics(usage_summary_data)

In [10]:
## Matplotlib
# def plot_usage_statistics_simple(data):
#     # Extrair os valores relevantes dos dados
#     months = [entry['month'] for entry in data['bioRxiv content statistics']]
#     abstract_views = [int(entry['abstract_views']) for entry in data['bioRxiv content statistics']]
#     pdf_downloads = [int(entry['pdf_downloads']) for entry in data['bioRxiv content statistics']]

#     # Criar um gráfico de linha
#     plt.figure(figsize=(18, 9))
#     plt.plot(months, abstract_views, label='Visualizações de Resumo de Conteúdo', marker='o', linestyle='-', color='b')
#     plt.plot(months, pdf_downloads, label='Downloads de PDF', marker='s', linestyle='--', color='r')

#     # Configurar rótulos e título
#     plt.xlabel('Mês')
#     plt.ylabel('Contagem')
#     plt.title('Estatísticas de Uso do bioRxiv')
#     plt.xticks(rotation=45, fontsize=8)  # Rotação dos rótulos do eixo x para melhor legibilidade
#     plt.grid(True)

#     # Adicionar legenda
#     plt.legend()

#     # Exibir o gráfico
#     plt.tight_layout()
#     plt.show()

# # Exemplo de uso com os dados fornecidos
# usage_data = fetch_usage_summary_statistics()

# plot_usage_statistics(usage_data)